In [28]:
with open("rm.txt") as f:
    a = [i.split(" @")[0].replace("\n", "") for i in f.readlines()]
    a = [i.split("==")[0] for i in a]

In [ ]:
import subprocess
from tqdm import tqdm

for i in tqdm(a):
    bash_command = f"pip3 uninstall {i} -y"
    process = subprocess.Popen(bash_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

    print(output)


## EZKL Jupyter Notebook Demo


In [11]:
!ls
%cd ..

data_anal.ipynb  ezkl_antifraud.ipynb
/home/sedan/ZeroEnter/antifraud


In [12]:
import torch

torch.cuda.is_available()

True

In [13]:
import torch

num_devices = torch.cuda.device_count()

for i in range(num_devices):
    print(torch.cuda.get_device_name(i))

NVIDIA RTX A6000


In [14]:
import yaml
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter

def parse_args():
    method = "rgtan"
    yaml_file = "config/rgtan_cfg.yaml"

    # config = Config().get_config()
    with open(yaml_file) as file:
        args = yaml.safe_load(file)
    args['method'] = method
    return args

args = parse_args()

In [15]:
args

{'batch_size': 64,
 'hid_dim': 256,
 'lr': 0.003,
 'wd': 0.0001,
 'n_layers': 2,
 'dropout': [0.2, 0.1],
 'device': 'cuda:0',
 'early_stopping': 10,
 'n_fold': 5,
 'seed': 2023,
 'max_epochs': 15,
 'gated': True,
 'dataset': 'amazon',
 'test_size': 0.6,
 'nei_att_heads': {'yelp': 4, 'amazon': 5, 'S-FFSD': 9},
 'method': 'rgtan'}

In [16]:
# from methods.rgtan.rgtan_main import rgtan_main, loda_rgtan_data

# feat_data, labels, train_idx, test_idx, g, cat_features, neigh_features = loda_rgtan_data(args['dataset'], args['test_size'])


# rgtan_main(feat_data, g, train_idx, test_idx, labels, args, cat_features, neigh_features, nei_att_head=args['nei_att_heads'][args['dataset']])

In [17]:
ls 

config/               LICENSE   models/     requirements.txt
data/                 main.py   notebooks/  rm_clear.txt
feature_engineering/  methods/  README.md   rm.txt


In [18]:
import numpy

In [20]:
import os
from dgl.dataloading import MultiLayerFullNeighborSampler
from dgl.dataloading import NodeDataLoader
from torch.optim.lr_scheduler import MultiStepLR
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score
from scipy.io import loadmat
from tqdm import tqdm
# from . import *
from methods.rgtan.rgtan_lpa import load_lpa_subtensor
from methods.rgtan.rgtan_model import RGTAN

In [23]:
from methods.rgtan.rgtan_main import rgtan_main, loda_rgtan_data

feat_data, labels, train_idx, test_idx, g, cat_features, neigh_features = loda_rgtan_data(
    args['dataset'], args['test_size'])

no neighbohood feature used.


In [ ]:
# torch.autograd.set_detect_anomaly(True)
device = args['device']
graph = g.to(device)
oof_predictions = torch.from_numpy(
    np.zeros([len(feat_df), 2])).float().to(device)
test_predictions = torch.from_numpy(
    np.zeros([len(feat_df), 2])).float().to(device)
kfold = StratifiedKFold(
    n_splits=args['n_fold'], shuffle=True, random_state=args['seed'])

y_target = labels.iloc[train_idx].values
num_feat = torch.from_numpy(feat_df.values).float().to(device)
cat_feat = {col: torch.from_numpy(feat_df[col].values).long().to(
    device) for col in cat_features}

neigh_padding_dict = {}
nei_feat = []
if isinstance(neigh_features, pd.DataFrame):  # otherwise []
    # if null it is []
    nei_feat = {col: torch.from_numpy(neigh_features[col].values).to(torch.float32).to(
        device) for col in neigh_features.columns}

y = labels
labels = torch.from_numpy(y.values).long().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
for fold, (trn_idx, val_idx) in enumerate(kfold.split(feat_df.iloc[train_idx], y_target)):
    print(f'Training fold {fold + 1}')
    trn_ind, val_ind = torch.from_numpy(np.array(train_idx)[trn_idx]).long().to(
        device), torch.from_numpy(np.array(train_idx)[val_idx]).long().to(device)

    train_sampler = MultiLayerFullNeighborSampler(args['n_layers'])
    train_dataloader = NodeDataLoader(graph,
                                      trn_ind,
                                      train_sampler,
                                      device=device,
                                      use_ddp=False,
                                      batch_size=args['batch_size'],
                                      shuffle=True,
                                      drop_last=False,
                                      num_workers=0
                                      )
    val_sampler = MultiLayerFullNeighborSampler(args['n_layers'])
    val_dataloader = NodeDataLoader(graph,
                                    val_ind,
                                    val_sampler,
                                    use_ddp=False,
                                    device=device,
                                    batch_size=args['batch_size'],
                                    shuffle=True,
                                    drop_last=False,
                                    num_workers=0,
                                    )
#     model = RGTAN(in_feats=feat_df.shape[1],
#                   hidden_dim=args['hid_dim']//4,
#                   n_classes=2,
#                   heads=[4]*args['n_layers'],
#                   activation=nn.PReLU(),
#                   n_layers=args['n_layers'],
#                   drop=args['dropout'],
#                   device=device,
#                   gated=args['gated'],
#                   ref_df=feat_df.iloc[train_idx],
#                   cat_features=cat_feat,
#                   neigh_features=nei_feat,
#                   nei_att_head=nei_att_head).to(device)
#     lr = args['lr'] * np.sqrt(args['batch_size']/1024)
#     optimizer = optim.Adam(model.parameters(), lr=lr,
#                            weight_decay=args['wd'])
#     lr_scheduler = MultiStepLR(optimizer=optimizer, milestones=[
#                                4000, 12000], gamma=0.3)

#     earlystoper = early_stopper(
#         patience=args['early_stopping'], verbose=True)
#     start_epoch, max_epochs = 0, 2000
#     for epoch in range(start_epoch, args['max_epochs']):
#         train_loss_list = []
#         # train_acc_list = []
#         model.train()
#         for step, (input_nodes, seeds, blocks) in enumerate(train_dataloader):
#             # print(f"loading batch data...")
#             batch_inputs, batch_work_inputs, batch_neighstat_inputs, batch_labels, lpa_labels = load_lpa_subtensor(num_feat, cat_feat, nei_feat, neigh_padding_dict, labels,
#                                                                                                                    seeds, input_nodes, device, blocks)
#             # print(f"load {step}")

#             # batch_neighstat_inputs: {"degree":(|batch|, degree_dim)}

#             blocks = [block.to(device) for block in blocks]
#             train_batch_logits = model(
#                 blocks, batch_inputs, lpa_labels, batch_work_inputs, batch_neighstat_inputs)
#             mask = batch_labels == 2
#             train_batch_logits = train_batch_logits[~mask]
#             batch_labels = batch_labels[~mask]
#             # batch_labels[mask] = 0

#             train_loss = loss_fn(train_batch_logits, batch_labels)
#             # backward
#             optimizer.zero_grad()
#             train_loss.backward()
#             optimizer.step()
#             lr_scheduler.step()
#             train_loss_list.append(train_loss.cpu().detach().numpy())

#             if step % 10 == 0:
#                 tr_batch_pred = torch.sum(torch.argmax(train_batch_logits.clone(
#                 ).detach(), dim=1) == batch_labels) / batch_labels.shape[0]
#                 score = torch.softmax(train_batch_logits.clone().detach(), dim=1)[
#                     :, 1].cpu().numpy()
#                 try:
#                     print('In epoch:{:03d}|batch:{:04d}, train_loss:{:4f}, '
#                           'train_ap:{:.4f}, train_acc:{:.4f}, train_auc:{:.4f}'.format(epoch, step,
#                                                                                        np.mean(
#                                                                                            train_loss_list),
#                                                                                        average_precision_score(
#                                                                                            batch_labels.cpu().numpy(), score),
#                                                                                        tr_batch_pred.detach(),
#                                                                                        roc_auc_score(batch_labels.cpu().numpy(), score)))
#                 except:
#                     pass

#         # mini-batch for validation
#         val_loss_list = 0
#         val_acc_list = 0
#         val_all_list = 0
#         model.eval()
#         with torch.no_grad():
#             for step, (input_nodes, seeds, blocks) in enumerate(val_dataloader):
#                 batch_inputs, batch_work_inputs, batch_neighstat_inputs, batch_labels, lpa_labels = load_lpa_subtensor(num_feat, cat_feat, nei_feat, neigh_padding_dict, labels,
#                                                                                                                        seeds, input_nodes, device, blocks)

#                 blocks = [block.to(device) for block in blocks]
#                 val_batch_logits = model(
#                     blocks, batch_inputs, lpa_labels, batch_work_inputs, batch_neighstat_inputs)
#                 oof_predictions[seeds] = val_batch_logits
#                 mask = batch_labels == 2
#                 val_batch_logits = val_batch_logits[~mask]
#                 batch_labels = batch_labels[~mask]
#                 # batch_labels[mask] = 0
#                 val_loss_list = val_loss_list + \
#                     loss_fn(val_batch_logits, batch_labels)
#                 # val_all_list += 1
#                 val_batch_pred = torch.sum(torch.argmax(
#                     val_batch_logits, dim=1) == batch_labels) / torch.tensor(batch_labels.shape[0])
#                 val_acc_list = val_acc_list + val_batch_pred * \
#                     torch.tensor(
#                         batch_labels.shape[0])  # how many in this batch is right!
#                 val_all_list = val_all_list + \
#                     batch_labels.shape[0]  # how many val nodes
#                 if step % 10 == 0:
#                     score = torch.softmax(val_batch_logits.clone().detach(), dim=1)[
#                         :, 1].cpu().numpy()
#                     try:
#                         print('In epoch:{:03d}|batch:{:04d}, val_loss:{:4f}, val_ap:{:.4f}, '
#                               'val_acc:{:.4f}, val_auc:{:.4f}'.format(epoch,
#                                                                       step,
#                                                                       val_loss_list/val_all_list,
#                                                                       average_precision_score(
#                                                                           batch_labels.cpu().numpy(), score),
#                                                                       val_batch_pred.detach(),
#                                                                       roc_auc_score(batch_labels.cpu().numpy(), score)))
#                     except:
#                         pass

#         # val_acc_list/val_all_list, model)
#         earlystoper.earlystop(val_loss_list/val_all_list, model)
#         if earlystoper.is_earlystop:
#             print("Early Stopping!")
#             break
#     print("Best val_loss is: {:.7f}".format(earlystoper.best_cv))
#     test_ind = torch.from_numpy(np.array(test_idx)).long().to(device)
#     test_sampler = MultiLayerFullNeighborSampler(args['n_layers'])
#     test_dataloader = NodeDataLoader(graph,
#                                      test_ind,
#                                      test_sampler,
#                                      use_ddp=False,
#                                      device=device,
#                                      batch_size=args['batch_size'],
#                                      shuffle=True,
#                                      drop_last=False,
#                                      num_workers=0,
#                                      )
#     b_model = earlystoper.best_model.to(device)
#     b_model.eval()
#     with torch.no_grad():
#         for step, (input_nodes, seeds, blocks) in enumerate(test_dataloader):
#             # print(input_nodes)
#             batch_inputs, batch_work_inputs, batch_neighstat_inputs, batch_labels, lpa_labels = load_lpa_subtensor(num_feat, cat_feat, nei_feat, neigh_padding_dict, labels,
#                                                                                                                    seeds, input_nodes, device, blocks)

#             blocks = [block.to(device) for block in blocks]
#             test_batch_logits = b_model(
#                 blocks, batch_inputs, lpa_labels, batch_work_inputs, batch_neighstat_inputs)
#             test_predictions[seeds] = test_batch_logits
#             test_batch_pred = torch.sum(torch.argmax(
#                 test_batch_logits, dim=1) == batch_labels) / torch.tensor(batch_labels.shape[0])
#             if step % 10 == 0:
#                 print('In test batch:{:04d}'.format(step))
# mask = y_target == 2
# y_target[mask] = 0
# my_ap = average_precision_score(y_target, torch.softmax(
#     oof_predictions, dim=1).cpu()[train_idx, 1])


# print("NN out of fold AP is:", my_ap)
# b_models, val_gnn_0, test_gnn_0 = earlystoper.best_model.to(
#     'cpu'), oof_predictions, test_predictions

# test_score = torch.softmax(test_gnn_0, dim=1)[test_idx, 1].cpu().numpy()
# y_target = labels[test_idx].cpu().numpy()
# test_score1 = torch.argmax(test_gnn_0, dim=1)[test_idx].cpu().numpy()

# mask = y_target != 2
# test_score = test_score[mask]
# y_target = y_target[mask]
# test_score1 = test_score1[mask]

# print("test AUC:", roc_auc_score(y_target, test_score))
# print("test f1:", f1_score(y_target, test_score1, average="macro"))
# print("test AP:", average_precision_score(y_target, test_score))

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        return logits


circuit = MyModel()

# Train the model as you like here (skipped for brevity)




In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:


# After training, export to onnx (network.onnx) and create a data file (input.json)
x = 0.1*torch.rand(1,*[1, 28, 28], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                  x,                   # model input (or a tuple for multiple inputs)
                  model_path,            # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [ ]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

In [ ]:
res = ezkl.compile_model(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = ezkl.get_srs(srs_path, settings_path)

In [ ]:
# now generate the witness file

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path, settings_path = settings_path)
assert os.path.isfile(witness_path)

In [ ]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
        settings_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "evm",
        "single",
        settings_path,
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")